In [2]:
# importação de bibliotecas
import requests
import pandas as pd
import matplotlib.pyplot as plt

# Configuração de estilo dos gráficos
plt.style.use("seaborn-v0_8")

base_url = "https://legis.senado.gov.br/dadosabertos/"

## Parlamentares da 56ª Legislatura (2019 a 2023)

In [ ]:
url_senadores_legislatura = base_url + "senador/lista/legislatura/56.csv"
cvs_senadores_legistura = "_data/senadores_legistatura_56.csv"

params = {    
            #"uf": "RJ",
            #"participacao": "T",
            #"exercicio": "exercicio",
            "v": 4
        }

headers = {"accept": "txt/csv"}

with requests.get(url_senadores_legislatura, params, headers=headers, stream=True, timeout=60) as r:
    r.raise_for_status()
    with open(cvs_senadores_legistura, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)

print("Arquivo salvo com sucesso!")

Arquivo salvo com sucesso!


### Parlamentares da 56ª Legislatura por forma de tratamento

In [ ]:
col = "IdentificacaoParlamentar.SexoParlamentar"
parlamentares[col] = parlamentares[col].astype(str).str.strip()

contagem = (
    parlamentares
      .groupby(col, dropna=True)
      .size()
      .reset_index(name="quantidade")
      .sort_values("quantidade", ascending=False)
)

contagem.head(3)

,IdentificacaoParlamentar.SexoParlamentar,quantidade
1,Masculino,197
0,Feminino,48


### Detalhes de um Parlamentar

In [ ]:
codigos_parlamentares = parlamentares["IdentificacaoParlamentar.CodigoParlamentar"]

urls = [base_url + f"senador/{codigo_parlamentar}.csv" for codigo_parlamentar in codigos_parlamentares]


detalhes_parlamentares = [requests.get(url, headers=headers).json()["DetalheParlamentar"]["Parlamentar"]["IdentificacaoParlamentar"] for url in urls]

#response = requests.get(url[0], headers=headers)

#detalhe_parlamentar = response.json()
#detalhe_parlamentar = detalhe_parlamentar["DetalheParlamentar"]["Parlamentar"]["IdentificacaoParlamentar"]
#detalhe_parlamentar = pd.json_normalize(detalhe_parlamentar, sep=".")

detalhes_parlamentares
#detalhe_parlamentares = detalhe_parlamentar


#detalhe_parlamentares.append(detalhe_parlamentar)

#detalhe_parlamentares







[{'CodigoParlamentar': '5573',
  'NomeParlamentar': 'Abel Rebouças',
  'NomeCompletoParlamentar': 'Abel Rebouças São José',
  'SexoParlamentar': 'Masculino',
  'SiglaPartidoParlamentar': 'PDT'},
 {'CodigoParlamentar': '4981',
  'CodigoPublicoNaLegAtual': '916',
  'NomeParlamentar': 'Acir Gurgacz',
  'NomeCompletoParlamentar': 'Acir Marcos Gurgacz',
  'SexoParlamentar': 'Masculino',
  'UrlFotoParlamentar': 'http://www.senado.leg.br/senadores/img/fotos-oficiais/senador4981.jpg',
  'UrlPaginaParlamentar': 'http://www25.senado.leg.br/web/senadores/senador/-/perfil/4981',
  'UrlPaginaParticular': 'https://acirgurgacz.com.br/',
  'EmailParlamentar': 'sen.acirgurgacz@senado.leg.br',
  'SiglaPartidoParlamentar': 'PDT'},
 {'CodigoParlamentar': '5918',
  'NomeParlamentar': 'Adilson Gomes',
  'NomeCompletoParlamentar': 'Adilson Gomes Silva',
  'SexoParlamentar': 'Masculino'},
 {'CodigoParlamentar': '5625',
  'NomeParlamentar': 'Adilson Silva dos Santos',
  'NomeCompletoParlamentar': 'Adilson Silv

## Entes

In [4]:
url = base_url + "processo/entes"

params = { "participação" : "T" }

response = requests.get(url, params)

# Verificando resposta
if (response.status_code == 200):
    entes = pd.DataFrame(response.json())
    entes = entes[pd.isnull(entes["dataFim"])]
else:
    print("Tratar retorno diferente de 200")

entes

,id,sigla,nome,casa,siglaTipo,descricaoTipo,dataInicio,dataFim
0,7378493,None,1ª Câmara de Coordenação e Revisão -Direitos S...,None,ENTE_JURIDICO,Ente Jurídico - não categorizado,None,None
1,7378350,None,1ª Câmara de Coordenação e Revisão do Ministér...,None,ENTE_JURIDICO,Ente Jurídico - não categorizado,None,None
2,301,None,1º Vice-Presidente da Mesa do Congresso Nacion...,None,ENTE_JURIDICO,Ente Jurídico - não categorizado,None,None
3,7376109,None,"2ª Vara Cível, Família e Sucessões Inf. e Juve...",None,ENTE_JURIDICO,Ente Jurídico - não categorizado,None,None
4,7376147,None,"2ª Vara de Família, Sucessões, Órfãos, Interdi...",None,ENTE_JURIDICO,Ente Jurídico - não categorizado,None,None
...,...,...,...,...,...,...,...,...
11374,7380144,None,associação brasileira de reciclagem animal,None,ENTE_JURIDICO,Ente Jurídico - não categorizado,None,None
11375,7349474,None,e-PETRO,None,ENTE_JURIDICO,Ente Jurídico - não categorizado,None,None
11376,7349700,None,emgea,None,ENTE_JURIDICO,Ente Jurídico - não categorizado,None,None
11377,7377116,None,entidades representativas do setor vitivinícol...,None,ENTE_JURIDICO,Ente Jurídico - não categorizado,None,None


In [ ]:
url = base_url + "processo"

params = {
            #"numero": "",
            #"ano": "2025",
            #"idProcesso": "",
            #"codigoMateria": "",
            "tramitando": "S",
            #tramitouLegislaturaAtual": "S" 
            "sigla": "SUG",
            #"dataInicioApresentacao": "2024-01-01",
            #"dataFimApresentacao": "2024-12-31",
            

        }

response = requests.get(url, params=params)

if (response.status_code == 200):
    processos = pd.DataFrame(response.json())
else:
    print(f"Tratar retorno {response.status_code}")

processos

,id,codigoMateria,identificacao,casaIdentificadora,enteIdentificador,tipoConteudo,ementa,tipoDocumento,dataApresentacao,autoria,tramitando,ultimaInformacaoAtualizada,dataUltimaAtualizacao,urlDocumento
0,7388637,133210,SUG 12/2018,SF,CDH,Sugestão Legislativa,Institui o Estatuto do Trabalho e regulamenta ...,Sugestão,2018-05-10,"ALJT, Anamatra, ANPT, Sinait",Sim,EVENTO_LEGISLATIVO,2025-08-14T13:54:32.061,https://legis.senado.gov.br/sdleg-getter/docum...
1,7394544,133422,SUG 13/2018,SF,CDH,Sugestão Legislativa,Revoga as Leis 13.467/2017 e 13429/2017,Sugestão,2018-05-28,"NCST, CONTRATUH, CNTI, CSPB, CNTTT e CNTEEC",Sim,NaN,NaN,https://legis.senado.gov.br/sdleg-getter/docum...
2,7706254,134957,SUG 39/2018,SF,PLEN,Sugestão Legislativa,Institui no ambiente escolar a Semana Nacional...,Sugestão,2018-12-18,Jovem Senador Jovem Senadora Ana Beatriz Barro...,Sim,NaN,NaN,https://legis.senado.gov.br/sdleg-getter/docum...
3,7727050,135618,SUG 2/2019,SF,CDH,Sugestão Legislativa,Isenção do IRPF para Profissionais de Seguranç...,Sugestão,2019-02-14,Programa e-Cidadania,Sim,EVENTO_LEGISLATIVO,2025-05-08T17:36:02.076,https://legis.senado.gov.br/sdleg-getter/docum...
4,7727055,135619,SUG 3/2019,SF,CDH,Sugestão Legislativa,"Enfermeiros(as)com piso salarial de R$ 4.800,0...",Sugestão,2019-02-14,Programa e-Cidadania,Sim,EVENTO_LEGISLATIVO,2025-04-25T17:25:24.123,https://legis.senado.gov.br/sdleg-getter/docum...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,8856603,169354,SUG 7/2025,SF,CDH,Sugestão Legislativa,"Dispõe sobre criação de ""Lei Federal para a pr...",Sugestão,2025-06-30,Programa e-Cidadania,Sim,EVENTO_LEGISLATIVO,2025-08-14T14:46:48.846,https://legis.senado.gov.br/sdleg-getter/docum...
96,8860593,169458,SUG 8/2025,SF,CDH,Sugestão Legislativa,"Dispõe sobre ""Piso salarial e carga horária de...",Sugestão,2025-07-08,Programa e-Cidadania,Sim,EVENTO_LEGISLATIVO,2025-08-27T10:46:15.56,https://legis.senado.gov.br/sdleg-getter/docum...
97,8890480,170012,SUG 9/2025,SF,CDH,Sugestão Legislativa,Proíbe o abate do jumento (Equus asinus) em to...,Sugestão,2025-08-22,FORUM NACIONAL DE PROTECAO E DEFESA ANIMAL,Sim,EVENTO_LEGISLATIVO,2025-09-09T16:13:53.598,https://legis.senado.gov.br/sdleg-getter/docum...
98,8890490,170013,SUG 10/2025,SF,CDH,Sugestão Legislativa,"Dispõe sobre: ""Estatuto dos Cães e Gatos"".",Sugestão,2025-08-22,Instituto Arcanimal; Instituto Faço pelos Anim...,Sim,EVENTO_LEGISLATIVO,2025-08-25T18:09:04.37,https://legis.senado.gov.br/sdleg-getter/docum...


In [ ]:
# ================================
# 3. Transformação em DataFrame
# ================================
# A API retorna XML/JSON dependendo da rota; 
# aqui usamos JSON (quando disponível) ou convertemos.

# Obs.: para XML seria preciso usar xml.etree.ElementTree ou lxml

data = response.json()
materias = data['ListaMaterias']['Materias']['Materia']

# Criando DataFrame
df = pd.DataFrame(materias)

# Exibir primeiras linhas
df.head()


In [ ]:
# ================================
# 4. Limpeza e preparação
# ================================
# Selecionando variáveis de interesse
colunas = ["idMateria", "siglaCasa", "siglaTipo", "numero", "ano", "ementa", "descricaoSituacao"]
df = df[colunas]

# Ajustar tipos de dados
df["ano"] = df["ano"].astype(int)
df["numero"] = df["numero"].astype(int)

df.head()


In [ ]:
# ================================
# 5. Estatísticas descritivas
# ================================
print("Total de proposições:", len(df))
print("\nResumo estatístico:")
print(df.describe(include="all"))


In [ ]:
# ================================
# 6. Visualizações iniciais
# ================================

# Quantidade de proposições por tipo
df["siglaTipo"].value_counts().plot(kind="bar", figsize=(8,4))
plt.title("Distribuição por Tipo de Proposição")
plt.xlabel("Tipo")
plt.ylabel("Quantidade")
plt.show()

# Proposições por ano
df["ano"].value_counts().sort_index().plot(kind="line", marker="o", figsize=(8,4))
plt.title("Proposições por Ano")
plt.xlabel("Ano")
plt.ylabel("Quantidade")
plt.show()


In [ ]:
# ================================
# 7. Discussão preliminar (anotações)
# ================================
# - Identificar variáveis com dados ausentes.
print(df.isna().sum())

# - Refletir sobre possíveis problemas:
#   * Situações pouco padronizadas.
#   * Textos longos em 'ementa' -> análise qualitativa ou NLP.
#   * Discrepâncias entre anos com muitas/ poucas proposições.
